# Adopting a Bioinformatics Prototype to Find Well-Conserved and Antigenic Epitopes In Related Proteins.


### Prof. Silva:

This is a proof-of-concept of how the tool I have been tinkering with to finding distinct regions in proteins could be modified to find conserved and antigenic ones. 

The paper you referred me to mentions that "functionally required epitopes are highly conserved among heterologous viral strains and represent a key vulnerability that could be targeted during vaccine development".

Using this as a cue, I have taken (somewhat arbitrarily) two HIV protein sequences, ENV_HV1MN Envelope glycoprotein gp160 (P05877) and ENV_HV1VI Envelope glycoprotein gp160 Q9QSQ7 and asked a simple question:

##### "If I wanted to target a region of these proteins, how could I pick an epitope that is both highly conserved (broadly neutralizing) and antigenic (likely to produce an immune response at all)?"

The goal for this proof-of-concept will be to computationally aide a researcher in selecting antigens that are conserved between the two proteins and likely to be antigenic. 

##### Specifically: we will attempt to find a list of 15mers that score well in our system for conservation and antigenicity.

## ASH - Antigen Selection Heuristic (re-purposed)


This is a proof of concept for a bioinformatics tool to aid in epitope selection. It is a simple but practical algorithm originally conceived to find chemically distinct regions of similar proteins so antibodies could be created to act on one while providing the smallest chance of cross reactivity with the other. It can also be used to do the opposite by filtering the data differently.

We will define a scale to weigh individual residues against one another and the use it to compare kmers in one protein to their analog in another. 

In this prototype, the system uses hydrophilicity as an approximation for antigenicity. We assign points the amino acids as follows. 


                D,E,R,K          =   Philes  --->   +0.5 points

                L,A,F,Y,W,I,M,V  =   Phobes  --->   -0.5 points

                All others       =   Neutral --->   +0.0 points


The program will iterate through a pair of aligned sequences and apply the tool to the two amino acids as each index of the sequences. For example, given "TIDE" and "TYDE" it would compare T to T, I to Y, D to D, and E to E. The is would result in an overall mismatch score (it is framed in terms of mismatches due to its initial purpose of finding distinct regions).

## Visualizing the ASH Scale

We can visualize the scale thusly:

                 Hydrophobe ---------- Neutral ---------- Hydrophile 
                 -0.5                     0                     +0.5


A critical observer might notice that this means a "H" to "H" match is scored the same as a "T" to "H". While they are similar for our purposes, they clearly should not be considered completely the same. To avoid this (and avoid unnecessary function calls), we use a simple equality comparison and simply skip identical residues. This results in a indirectly assigned mismatch score of 0 for matches. Once we do this, we can maintain our simple scoring. If the absolute value of the comparison results in a 0, a score of 0.25 is returned by default. This means residues from the same category (that aren't identical) receive the smallest penalty possible, and that penalty will increase by 0.25 with each consecutive step "away" from one another they are. Thus our scale ranges from -0.5 to + 0.5 in increments of 0.25. If there is a gap in the sequence, we will assign a mismatch score of 2 to reflect that even very different residues are closer together than a match between a residue and nothing. This thinking is based on maintaining the simplicity of the tool and the fact that gaps are weighted heavily in the alignment algorithms.

            H <--> H = 0.00 : Identical, function not called
            
            T <--> H = 0.25 : Neutral <--> Neutral - score would be 0, return 0.25
            
            H <--> D = 0.50 : Neutral <--> Phile - One "step" away, return 0.50
            
            Y <--> Y = 1.00 : Phobe   <--> Phile - Two "steps" away, return 1.00
    

To illustrate visually:

                                      A Match:
                         Function not called,Defaults to zero 
                                 
                                 
                                 


                                        H:H
                 Hydrophobe|-----------Neutral-----------|Hydrophile 
                 -0.5                    0                      +0.5


                             Similar but not identical:
                                  Defaults to 0.25

                                     H <-----> T
                 Hydrophobe|-----------Neutral-----------|Hydrophile 
                 -0.5                     0                     +0.5


                                 Neutral to Phile:

                                          H <--------------------> D
                 Hydrophobe|-----------Neutral-----------|Hydrophile 
                 -0.5                     0                     +0.5




                                  Phobe to Phile:

                 Y <---------------------------------------------> D
                 Hydrophobe|-----------Neutral-----------|Hydrophile 
                 -0.5                     0                     +0.5


### Coding the first function 

We can write a function that implements this scoring scheme using a dictionary. Recall that for the purposes of this prototype, we will give a "-" in an alignment a score of two, implying that having no residue at the index in one sequence makes it twice as distinct as having a residue there that is very different. 

In [76]:
import math

"""This function finds the score of two residues """

def weighted_score(residue1, residue2):
    # hydrophiles are positive, hydrophobic is negative, neutral is 0 
    weight = {  "L":-0.5, "A":-0.5, "F":-0.5, "Y":-0.5, "W":-0.5,
                "I":-0.5, "V":-0.5, "H":+0.0, "N":+0.0, "C":+0.0,
                "G":+0.0, "M":+0.0, "Q":+0.0, "P":+0.0, "S":+0.0, 
                "T":+0.0, "D":+0.5, "E":+0.5, "R":+0.5, "K":+0.5 }
    
    # a gap is given a score of two in our system 
    if residue1 == "-" or residue2 == "-":
        return 2.0
    
    # subscore is the abs value of the scores 
    subscore = abs(weight[residue1] - weight[residue2])
    
    # same group returns 0.25
    if subscore == 0:
        return 0.25
    else:
        return subscore

Next we will make a function that applies the above function to each letter at the same index in a pair of sequences. 

In [77]:

"""This functions scores peptides using the scale and function above"""

# create a scoring function
def mismatch(input_seq1, input_seq2):
    score = 0
    # for each residue in the sequences
    for i in range(len(input_seq1)):
        # identical give no score
        if input_seq1[i] == input_seq2[i]:
            score += 0
        else:
            # use scoring function for all non-matches
            score += weighted_score(input_seq1[i], input_seq2[i])
    return score



Here is an example of the function in use:


In [78]:

# examples sequence
pep1 = "PEPTIDE"
pep2 = "PEPTYDE"

# iterate throught he sequences and call the function 
for i in range(len(pep1)):
    print(mismatch(pep1[i], pep2[i]))

0
0
0
0
0.25
0
0


This is what we would expect: they only differ in one point, by a residue we consider "similar but not identical".

## Application to Full Peptides


Now we can write a function that applies this to a sequence as a whole. The result of this function will be the "mismatch" score, of the two peptides.

We can now apply that to the sample peptides mentioned earlier.

In [79]:
print(mismatch(pep1, pep2))

0.25


Another example to drive it home. 

In [80]:

# try on new peptides 
pep3 = "PEPTIDE"
pep4 = "PEPTAAE"

print(mismatch(pep3, pep4))

1.25


To see how this works in practice we will open fasta files storing information on ENV_HV1MN Envelope glycoprotein gp160 and ENV_HV1VI Envelope glycoprotein gp160. First, a quick parser for single entry fasta files:

## Testing it on Our Sequences 

In [81]:

# this function extracts the sequence from a fasta file
def get_seq(filename):
    seq = ''
    # read in file, skp header, save sequences
    data = open(filename, 'r').readlines()
    for line in data:
        if not line.startswith('>'): # skips header
            line = line.replace('\n', '')
            seq += line
    return seq

Let's use the parse to read in our sequences:

In [82]:

# get our files using the new function 
ENV_HV1MN = get_seq("ENV_HV1MN.fasta")
ENV_HV1VI = get_seq("ENV_HV1VI.fasta")

# print one to see if it is working 
print(ENV_HV1MN)

MRVKGIRRNYQHWWGWGTMLLGLLMICSATEKLWVTVYYGVPVWKEATTTLFCASDAKAYDTEVHNVWATQACVPTDPNPQEVELVNVTENFNMWKNNMVEQMHEDIISLWDQSLKPCVKLTPLCVTLNCTDLRNTTNTNNSTANNNSNSEGTIKGGEMKNCSFNITTSIRDKMQKEYALLYKLDIVSIDNDSTSYRLISCNTSVITQACPKISFEPIPIHYCAPAGFAILKCNDKKFSGKGSCKNVSTVQCTHGIRPVVSTQLLLNGSLAEEEVVIRSENFTDNAKTIIVHLNESVQINCTRPNYNKRKRIHIGPGRAFYTTKNIIGTIRQAHCNISRAKWNDTLRQIVSKLKEQFKNKTIVFNQSSGGDPEIVMHSFNCGGEFFYCNTSPLFNSTWNGNNTWNNTTGSNNNITLQCKIKQIINMWQEVGKAMYAPPIEGQIRCSSNITGLLLTRDGGKDTDTNDTEIFRPGGGDMRDNWRSELYKYKVVTIEPLGVAPTKAKRRVVQREKRAAIGALFLGFLGAAGSTMGAASVTLTVQARLLLSGIVQQQNNLLRAIEAQQHMLQLTVWGIKQLQARVLAVERYLKDQQLLGFWGCSGKLICTTTVPWNASWSNKSLDDIWNNMTWMQWEREIDNYTSLIYSLLEKSQTQQEKNEQELLELDKWASLWNWFDITNWLWYIKIFIMIVGGLVGLRIVFAVLSIVNRVRQGYSPLSLQTRPPVPRGPDRPEGIEEEGGERDRDTSGRLVHGFLAIIWVDLRSLFLFSYHHRDLLLIAARIVELLGRRGWEVLKYWWNLLQYWSQELKSSAVSLLNATAIAVAEGTDRVIEVLQRAGRAILHIPTRIRQGLERALL


The sequences are not of identical length, so we will have to align them. We will use skbio's Smith-Waterman implementation to achieve this:

In [83]:
from skbio.alignment import StripedSmithWaterman

# make a query object of the first protein 
query = StripedSmithWaterman(ENV_HV1MN)

# use it to create an alignment
alignment = query(ENV_HV1VI)
print(alignment)

GIRRNYQHWW...
GMQRNWQHLG...
Score: 185
Length: 853


We can now extract aligned sequences and the gaps will be filled appropriately​. 

In [84]:

# extract the sequence 
HV1MN = alignment.aligned_query_sequence
HV1VI = alignment.aligned_target_sequence

print("HV1MN:\n" + HV1MN + "\n")
print("HV1VI:\n" + HV1VI)

HV1MN:
GIRRNYQHWWGWGTMLLGLLMICSATEKLWVTVYYGVPVWKEATTTLFCASDAKAYDTEVHNVWATQACVPTDPNPQEVELVNVTENFNMWKNNMVEQMHEDIISLWDQSLKPCVKLTPLCVTLNCTDLRNTTNTNNSTANNNSNSEGTIKGGEMKNCSFNITTSIRDKMQKEYALLYKLDIVSIDNDSTSYRLISCNTSVITQACPKISFEPIPIHYCAPAGFAILKCNDKKFSGKGSCKNVSTVQCTHGIRPVVSTQLLLNGSLAEEEVVIRSENFTDNAKTIIVHLNESVQINCTRPNYNKRKRIHIGPGRAFYTTKNIIGTIRQAHCNISRAKWNDTLR-QIVSKLKEQFKNKTIVFNQSSGGDPEIVMHSFNCGGEFFYCNTSPLFNSTWNGNNTWNNTTGSNNNITLQCKIKQIINMWQEVGKAMYAPPIEGQIRCSSNITGLLLTRDGGKDTDTNDTEIFRPGGGDMRDNWRSELYKY-KVVTIEPLGVAPTKAKRRVVQREKRAAIGALFLGFLGAAGSTMGAASVTLTVQARLLLSGIVQQQNNLLRAIEAQQHMLQLTVWGIKQLQARVLAVERYLKDQQLLGFWGCSGKLICTTTVPWNASWSNKSLDDIWNNMTWMQWEREIDNYTSLIYSLLEKSQTQQEKNEQELLELDKWASLWNWFDITNWLWYIKIFIMIVGGLVGLRIVFAVLSIVNRVRQGYSPLSLQTRPPVPRGP-DRPEGIEEEGGERDRDTSGRLVHGFLAIIWVDLRSLFLFSYHHRDLLLIAARIVELLGRRGWEVLKYWWNLLQYWSQELKSSAVSLLNATAIAVAEGTDRVIEVLQRAGRAILHIPTRIRQGLERA

HV1VI:
GMQRNWQHLGKWGLLFLGILIICNAADNLWVTVYYGVPVWKEATTTLFCASDAKAYEREAHNVWATHACVPTDPNPQEVFLKNVTENFDMWKNNMVEQMHTDIISLWDQSLKPCVKLTPLCVTLNCTNATN

We can now find out how distinct there are in terms of our scale:

In [85]:
print(mismatch(HV1MN, HV1VI))

179.25


## Putting it All Together 

Alone, this is not of much use. To gain insight form this technique, we must apply it to every kmer in a protein. Let's say we are looking for epitopes 15mers long. We will compare each 15mer in the first protein to it's analog in the other, thus finding which kmers are the most distinct. We will define a custom data structure that holds the sequence of the peptide, the position it occurs at in the seqence, the score of the peptide, the peptide it was compared to, and a simple measure of antigenicity.

In [86]:

"""This class stores the data for each comparison"""

class Entry(object):
    
    def __init__(self, seq, pos, score, match, antg):
        self.seq   =   seq   # the peptide
        self.pos   =   pos   # what index is appears
        self.score = score   # the mismatch score
        self.match = match   # what it was compared to
        self.antg  = antg    # the simple antigenicty score

The antigenicity heuristic will use hydrophilicty as its basis. Later versions of this function will include nuances such as the complexity of the peptide.

In [87]:

""" This function implements a simple antigenicity heuristic"""

def antigenicity(seq):
    # count occurences of antigenic residues 
    simple_scores = ["D","E","R","K"]
    score = 0
    for item in seq:
        if item in simple_scores:
            score += 1
    return round(score/len(seq), 2)

We're now ready to implement a sequence to sequence comparison function. It will iterate through the sequences, make the comparison, and store the data in the Entry class. This function will return a list of Entry objects that we can parse through.

In [88]:

"""This function compares two sequences, calling the mistmatch at each location"""

# define a seq-seq comparison function
def seq_to_seq(seq1, seq2, length):
    
    # we'll store a list of objects
    results = []       
    
    # we start at 0
    position = 0
    
    # iterate up to the end of the seqs
    while position + length <= len(seq1):
        
        # isilate the region on each protein
        current_peptide = seq1[position:position+length]
        compare_peptide = seq2[position:position+length]
        
        # analyze those kmers 
        entry = mismatch(current_peptide, compare_peptide)
        
        # create and object to store the results
        results_obj = Entry(
                            seq   = current_peptide,
                            pos   = position,
                            score = entry,
                            match = compare_peptide,
                            antg  = antigenicity(current_peptide)
                            )
        
        # capture objects created
        results.append(results_obj)
        
        # increse the step
        position += 1
    return results

This will allow us to see the properties of individual regions of the proteins. we call the sequence to sequence function on our aligned proteins, passing the instruction to look for epitopes 15 residues long. 

In [89]:

# call the function on our aligned proteins 
results = seq_to_seq(HV1MN,HV1VI, 15)

## Now What? Narrowing Down The Search

There are 839 objects in this list. For this to be useful, we will need to be able to apply some method of filtration. Fortunately, we can.

The paper "Targeted Elimination of Immunodominant B Cells Drives the Germinal Center Reaction toward Subdominant Epitopes", the inspiration behind the adoption of this tool, describes a search for subdominant epitopes that might be broadly useful against HIV, and describes a method for inhibition of the dominant epitopes to permit the development of subdominant epitopes. When discussing the potential role of bioinformatics in this goal, Prof. Sitkovsky mentioned the desire to "select the best peptide among those in the target area". This tool aims to equip the researchers to do so.

It stands to reason that a good target in this context would need to be well conserved and also antigenic.

Using this function, we can attempt to see what epitopes are both well conserved and likely to antigenic. For sake of argument, let's assume we want a 15mer in the first protein that has a mismatch of no more than 0.5 on our scale (meaning it is well conserved), but also has an antigenicity score of at least .2 (20% hyrdophilic residues). 

In [90]:

#print out a simple header
print("Index\t","Seq\t\t\t", "Score","  Atng", "\t Compared to")

# iterate though the results
for item in results:
    # if the score < 0.5 and the antigenicity > 25%
    if item.score < 0.5 and item.antg > 0.25:
        # print out the results 
        print(item.pos, "\t", item.seq,"\t",item.score,
              "\t", item.antg, "\t",item.match, "\t")
    

Index	 Seq			 Score   Atng 	 Compared to
39 	 WKEATTTLFCASDAK 	 0 	 0.27 	 WKEATTTLFCASDAK 	
40 	 KEATTTLFCASDAKA 	 0 	 0.27 	 KEATTTLFCASDAKA 	
101 	 DIISLWDQSLKPCVK 	 0 	 0.27 	 DIISLWDQSLKPCVK 	
569 	 WGIKQLQARVLAVER 	 0 	 0.27 	 WGIKQLQARVLAVER 	
570 	 GIKQLQARVLAVERY 	 0 	 0.27 	 GIKQLQARVLAVERY 	
571 	 IKQLQARVLAVERYL 	 0 	 0.27 	 IKQLQARVLAVERYL 	
572 	 KQLQARVLAVERYLK 	 0 	 0.33 	 KQLQARVLAVERYLK 	
573 	 QLQARVLAVERYLKD 	 0 	 0.33 	 QLQARVLAVERYLKD 	
574 	 LQARVLAVERYLKDQ 	 0 	 0.33 	 LQARVLAVERYLKDQ 	
575 	 QARVLAVERYLKDQQ 	 0 	 0.33 	 QARVLAVERYLKDQQ 	
576 	 ARVLAVERYLKDQQL 	 0 	 0.33 	 ARVLAVERYLKDQQL 	
577 	 RVLAVERYLKDQQLL 	 0 	 0.33 	 RVLAVERYLKDQQLL 	
578 	 VLAVERYLKDQQLLG 	 0 	 0.27 	 VLAVERYLKDQQLLG 	
579 	 LAVERYLKDQQLLGF 	 0.25 	 0.27 	 LAVERYLKDQQLLGI 	
580 	 AVERYLKDQQLLGFW 	 0.25 	 0.27 	 AVERYLKDQQLLGIW 	
581 	 VERYLKDQQLLGFWG 	 0.25 	 0.27 	 VERYLKDQQLLGIWG 	
582 	 ERYLKDQQLLGFWGC 	 0.25 	 0.27 	 ERYLKDQQLLGIWGC 	
820 	 VAEGTDRVIEVLQRA 	 0.25 	 0.33 	 VAEGTDRIIE

Looking at this readout, it is readily apparent that kmers starting in the region 572-577 are totally conserved (mismatch = 0) in both proteins, so looking in the regions 572-592 on the first protein would likely be a good place to start. Also, with the cost of only 0.25 mismatch points, the kmer starting at index 822 (EGTDRVIEVLQRAGR), is almost half highly antigenic regions. 

While this is a small example on two proteins, it could be readily applied again to different samples, as simply as reading in two new fastas, and could theoretically be applies to larger sets of proteins with some modifications.

### Note:
Since the was written new functions have been added to find percent hydrophilic residues, percent structurally complex (aromatic) residues, and a simple weighted score of their mismatch of complex residues. 

Thank you for reading!

Thadryan 